In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pyart

In [ ]:
"""
pyart.retrieve.quasi_vertical_profile
=====================================

Retrieval of QVPs from a radar object

.. autosummary::
    :toctree: generated/

    quasi_vertical_profile

"""

import numpy as np

from pyart.core import antenna_to_cartesian



def quasi_vertical_profile(radar, fields = None, gatefilter = None):
    
    """
    Quasi Vertical Profile.
    
    Creates a QVP object containing fields from a radar object that can
    be used to plot and produce the quasi vertical profile.
    
    
    Parameters
    ----------
    radar : Radar
        Radar object used.
    field : string
        Radar field to use for QVP calculation.
    
    Other Parameters
    ----------------
    gatefilter : GateFilter
        A GateFilter indicating radar gates that should be excluded
        from the import qvp calculation
        
    Returns
    -------
    qvp : Dictonary
        Dictonary containing fields from radar object
    
    
    
    References
    ----------
    Troemel, S., M. Kumjian, A. Ryzhkov, and C. Simmer, 2013: Backscatter 
    differential phase – estimation and variability. J. Appl. Meteor. Clim.. 
    52, 2529 – 2548.

    Troemel, S., A. Ryzhkov, P. Zhang, and C. Simmer, 2014: Investigations 
    of backscatter differential phase in the melting layer. J. Appl. Meteorol. 
    Clim. 53, 2344 – 2359.

    Ryzhkov, A., P. Zhang, H. Reeves, M. Kumjian, T. Tschallener, S. Troemel, 
    C. Simmer, 2015: Quasi-vertical profiles – a new way to look at polarimetric 
    radar data. Submitted to J. Atmos. Oceanic Technol.
    
    """
    
    qvp = {}
    desired_angle = 20.0
    index = abs(radar.fixed_angle['data'] - desired_angle).argmin()
    print(radar.fixed_angle['data'])
    print(radar.elevation['data'][-1])
    
    radar_slice = radar.get_slice(index)
    
    if fields is None:
        fields = radar.fields

        for field in fields:
            if gatefilter is not None:
                get_fields = radar.get_field(index, field)
                mask_fields = np.ma.masked_where(gatefilter.gate_excluded[radar_slice], 
                                                get_fields)
                radar_field = np.ma.mean(mask_fields, axis = 0)
            else: 
                radar_fields = radar.get_field(index, field).mean(axis = 0)
            
            qvp.update({field:radar_fields})

    else:
        if gatefilter is not None:
            get_field = radar.get_field(index, fields)
            mask_field = np.ma.masked_where(gatefilter.gate_excluded[radar_slice], 
                                            get_field)
            radar_field = np.ma.mean(mask_field, axis = 0)
        else: 
            radar_field = radar.get_field(index, fields).mean(axis = 0)
            
        qvp.update({fields:radar_field})
        
    qvp.update({'range': radar.range['data'], 'time': radar.time})
    x,y,z = antenna_to_cartesian(qvp['range']/1000.0, 0.0, 
                                            radar.fixed_angle['data'][index])
    qvp.update({'height': z})
    del radar
    return qvp

In [ ]:
"""Unit Tests for Py-ART's retrieve/qvp.py module."""
from numpy.testing import assert_almost_equal

In [ ]:
def test_quasi_vertical_profile():
    test_radar = pyart.testing.make_target_radar()
    height = np.arange(0, 1000, 200)
    speed = np.ones_like(height) * 5
    direction = np.array([0, 90, 180, 270, 45])
    profile = pyart.core.HorizontalWindProfile(height, speed, direction)
    sim_vel = pyart.util.simulated_vel_from_profile(test_radar, profile)
    test_radar.add_field('velocity', sim_vel,
                         replace_existing=True)
    qvp = quasi_vertical_profile(test_radar)

    
    qvp_height = [0., 0., 0., 1., 1., 1., 1., 2., 2., 2., 2., 3.,
                  3., 3., 3., 3., 4., 4., 4., 4., 5., 5., 5., 5.,
                  6., 6., 6., 7., 7., 7., 7., 8., 8., 8., 8., 9.,
                  9., 9., 10., 10., 10., 10., 11., 11., 11., 11., 
                  12., 12., 12., 13.]
    
    qvp_range = [ 0., 20.408163, 40.816326, 61.22449, 81.63265, 102.04082, 
                 122.44898, 142.85715, 163.2653, 183.67346, 204.08163, 224.48979, 
                 244.89796, 265.30612, 285.7143, 306.12244, 326.5306, 346.93878, 
                 367.34692, 387.7551, 408.16327, 428.57144, 448.97958, 469.38776, 
                 489.79593, 510.20407, 530.61224, 551.0204, 571.4286, 591.83673,
                 612.2449, 632.6531, 653.0612, 673.46936, 693.87756, 714.2857,
                 734.69385, 755.10205, 775.5102, 795.9184, 816.32654, 836.7347,
                 857.1429, 877.551, 897.95917, 918.3674, 938.7755, 959.18365, 
                 979.59186, 1000]
    
    qvp_reflectivity = [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10., 10., 10., 
                        10., 10., 10., 10., 10., 10., 10., 20., 20., 20., 20., 20., 20., 
                        20., 20., 20., 20., 30., 30., 30., 30., 30., 30., 30., 30., 30., 
                        30., 40., 40., 40., 40., 40., 40., 40., 40., 40., 40.]
    
    qvp_velocity = [7.448253646563035e-09, 7.448253646563035e-09, 7.448253646563035e-09,
                    7.212727631475402e-09, 7.212727631475402e-09, 7.212727631475402e-09,
                    7.212727631475402e-09, 6.977201207147226e-09, 6.977201207147226e-09,
                    6.977201207147226e-09, 6.977201207147226e-09, 6.741673554133687e-09,
                    6.741673554133687e-09, 6.741673554133687e-09, 6.741673554133687e-09,
                    6.741673554133687e-09, 6.5061473227453115e-09, 6.5061473227453115e-09,
                    6.5061473227453115e-09, 6.5061473227453115e-09, 6.270622495326468e-09,
                    6.270622495326468e-09, 6.270622495326468e-09, 6.270622495326468e-09,
                    6.0350943047028496e-09, 6.0350943047028496e-09, 6.0350943047028496e-09,
                    5.799567182514694e-09, 5.799567182514694e-09, 5.799567182514694e-09,
                    5.799567182514694e-09, 5.5640389605889535e-09, 5.5640389605889535e-09,
                    5.5640389605889535e-09, 5.5640389605889535e-09, 5.328510501969833e-09,
                    5.328510501969833e-09, 5.328510501969833e-09, 5.092988902216039e-09,
                    5.092988902216039e-09, 5.092988902216039e-09, 5.092988902216039e-09,
                    4.857459394590356e-09, 4.857459394590356e-09, 4.857459394590356e-09,
                    4.857459394590356e-09, 4.621931789300988e-09, 4.621931789300988e-09,
                    4.621931789300988e-09, 4.386406373001349e-09]
    
    assert_almost_equal(qvp['height'], qvp_height, 3)
    assert_almost_equal(qvp['range'], qvp_range, 3)
    assert_almost_equal(qvp['reflectivity'], qvp_reflectivity, 3)
    assert_almost_equal(qvp['velocity'], qvp_velocity, 3)

In [ ]:
test_quasi_vertical_profile()

In [ ]:
test_radar = pyart.testing.make_target_radar()
height = np.arange(0, 1000, 200)
speed = np.ones_like(height) * 5
direction = np.array([0, 90, 180, 270, 45])
profile = pyart.core.HorizontalWindProfile(height, speed, direction)
sim_vel = pyart.util.simulated_vel_from_profile(test_radar, profile)
test_radar.add_field('velocity', sim_vel,
                     replace_existing=True)
qvp = quasi_vertical_profile(test_radar)

print(qvp['height'])
print(qvp['range'])
print(qvp['reflectivity'])
print(qvp['velocity'])